<a href="https://colab.research.google.com/github/Capstone-Team-CH2-PS379/Machine-Learning/blob/main/model-experiments/Fine_tuning_Wav2Vec2.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

# Setting Up

In [1]:
!pip3 install -q git+https://github.com/vasudevgupta7/gsoc-wav2vec2@main
!sudo apt-get install -y libsndfile1-dev
!pip3 install -q SoundFile

  Preparing metadata (setup.py) ... done
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 2.1/2.1 MB 9.6 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 3.3/3.3 MB 18.8 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 190.6/190.6 kB 19.0 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 254.1/254.1 kB 23.2 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 62.7/62.7 kB 6.8 MB/s eta 0:00:00
ERROR: pip's dependency resolver does not currently take into account all the packages that are installed. This behaviour is the source of the following dependency conflicts.
tensorflow-probability 0.22.0 requires typing-extensions<4.6.0, but you have typing-extensions 4.9.0 which is incompatible.
Reading package lists... Done
Building dependency tree... Done
Reading state information... Done
libsndfile1-dev is already the newest version (1.0.31-2ubuntu0.1).
0 upgraded, 0 newly installed, 0 to remove and 23 not upgraded.


# Model Setup

In [35]:
import os
import tensorflow as tf
import tensorflow_hub as hub
from wav2vec2 import Wav2Vec2Config

print("TF version:", tf.__version__)

TF version: 2.14.0


In [184]:
config = Wav2Vec2Config()
pretrained_layer = hub.KerasLayer("https://tfhub.dev/vasudevgupta7/wav2vec2/1", trainable=True)

In [232]:
# Hyperparameters setup

audio_maxlen = 246000   #Model only accepts static sequence length of 246000
label_maxlen = 256
batch_size = 2

In [185]:
inputs = tf.keras.Input(shape=(audio_maxlen,))
hidden_states = pretrained_layer(inputs)
outputs = tf.keras.layers.Dense(config.vocab_size)(hidden_states)

model = tf.keras.Model(inputs=inputs, outputs=outputs)

In [186]:
# Setting up training state

model(tf.random.uniform(shape=(batch_size, audio_maxlen)))
model.summary()

Model: "model_1"
_________________________________________________________________
 Layer (type)                Output Shape              Param #   
 input_2 (InputLayer)        [(None, 246000)]          0         
                                                                 
 keras_layer_2 (KerasLayer)  (None, 768, 768)          94371712  
                                                                 
 dense_1 (Dense)             (None, 768, 32)           24608     
                                                                 
Total params: 94396320 (360.09 MB)
Trainable params: 94396320 (360.09 MB)
Non-trainable params: 0 (0.00 Byte)
_________________________________________________________________


In [187]:
# Loss Function and Optimizer
from wav2vec2 import CTCLoss

learning_rate = 5e-5
loss_fn = CTCLoss(config, (batch_size, audio_maxlen), division_factor=batch_size)
optimizer = tf.keras.optimizers.Adam(learning_rate)

# Load Data

In [9]:
# We will use LibriSpeech dataset
!wget https://www.openslr.org/resources/12/dev-clean.tar.gz -P ./data/train/
!tar -xf ./data/train/dev-clean.tar.gz -C ./data/train/

--2023-12-14 16:40:39--  https://www.openslr.org/resources/12/dev-clean.tar.gz
Resolving www.openslr.org (www.openslr.org)... 46.101.158.64
Connecting to www.openslr.org (www.openslr.org)|46.101.158.64|:443... connected.
HTTP request sent, awaiting response... 302 Found
Location: http://us.openslr.org/resources/12/dev-clean.tar.gz [following]
--2023-12-14 16:40:40--  http://us.openslr.org/resources/12/dev-clean.tar.gz
Resolving us.openslr.org (us.openslr.org)... 46.101.158.64
Connecting to us.openslr.org (us.openslr.org)|46.101.158.64|:80... connected.
HTTP request sent, awaiting response... 200 OK
Length: 337926286 (322M) [application/x-gzip]
Saving to: ‘./data/train/dev-clean.tar.gz’

dev-clean.tar.gz    100%[===================>] 322.27M  28.4MB/s    in 12s     

2023-12-14 16:40:52 (26.2 MB/s) - ‘./data/train/dev-clean.tar.gz’ saved [337926286/337926286]



In [45]:
!wget https://www.openslr.org/resources/12/train-clean-100.tar.gz -P ./data/train/
!tar -xf ./data/train/train-clean-100.tar.gz -C ./data/train/

--2023-12-14 17:20:05--  https://www.openslr.org/resources/12/train-clean-100.tar.gz
Resolving www.openslr.org (www.openslr.org)... 46.101.158.64
Connecting to www.openslr.org (www.openslr.org)|46.101.158.64|:443... connected.
HTTP request sent, awaiting response... 302 Found
Location: http://us.openslr.org/resources/12/train-clean-100.tar.gz [following]
--2023-12-14 17:20:05--  http://us.openslr.org/resources/12/train-clean-100.tar.gz
Resolving us.openslr.org (us.openslr.org)... 46.101.158.64
Connecting to us.openslr.org (us.openslr.org)|46.101.158.64|:80... connected.
HTTP request sent, awaiting response... 200 OK
Length: 6387309499 (5.9G) [application/x-gzip]
Saving to: ‘./data/train/train-clean-100.tar.gz’

train-clean-100.tar 100%[===================>]   5.95G  28.2MB/s    in 3m 33s  

2023-12-14 17:23:39 (28.6 MB/s) - ‘./data/train/train-clean-100.tar.gz’ saved [6387309499/6387309499]



In [188]:
# File list inside the directory
print(os.listdir('/content/data/train'))

['dev-clean.tar.gz', 'LibriSpeech', 'train-clean-100.tar.gz']


In [227]:
base_dir = "./data/train/LibriSpeech/train-clean-100"
txt_files = []
flac_files = []

for subdir, dirs, files in os.walk(base_dir):
    for file in files:
        file_path = os.path.join(subdir, file)
        if file.endswith(".txt"):
            txt_files.append(file)
        elif file.endswith(".flac"):
            flac_files.append(file)

print("Transcription files:", txt_files)
print("Sound files:", flac_files)
print(len(txt_files))
print(len(flac_files))

Transcription files: ['118-124588.trans.txt', '118-121721.trans.txt', '118-47824.trans.txt', '2092-145709.trans.txt', '2092-145706.trans.txt', '7794-295947.trans.txt', '7794-295955.trans.txt', '7794-295948.trans.txt', '1578-140049.trans.txt', '1578-6379.trans.txt', '1578-140045.trans.txt', '7278-91083.trans.txt', '7278-104730.trans.txt', '7278-246956.trans.txt', '3807-4955.trans.txt', '3807-4923.trans.txt', '200-126784.trans.txt', '200-124139.trans.txt', '200-124140.trans.txt', '3857-180923.trans.txt', '3857-182317.trans.txt', '3857-182315.trans.txt', '2952-407.trans.txt', '2952-408.trans.txt', '2952-410.trans.txt', '8108-274318.trans.txt', '8108-280359.trans.txt', '8108-280354.trans.txt', '730-359.trans.txt', '730-358.trans.txt', '730-360.trans.txt', '1034-121119.trans.txt', '4813-248641.trans.txt', '4813-248638.trans.txt', '226-131533.trans.txt', '226-131532.trans.txt', '226-122538.trans.txt', '3259-158083.trans.txt', '6531-61334.trans.txt', '3982-178459.trans.txt', '3982-182255.tran

In [190]:
# Load text data

def read_txt_file(f):
  with open(f, "r") as f:
    samples = f.read().split("\n")
    samples = {s.split()[0]: " ".join(s.split()[1:]) for s in samples if len(s.split()) > 2}
  return samples

In [191]:
# Load speech data

import soundfile as sf
required_sample_rate = 16000

def read_flac_file(file_path):
  with open(file_path, "rb") as f:
      audio, sample_rate = sf.read(f)
  if sample_rate != required_sample_rate:
      raise ValueError(
          f"sample rate (={sample_rate}) of your files must be {required_sample_rate}"
      )
  file_id = os.path.split(file_path)[-1][:-len(".flac")]
  return {file_id: audio}

In [192]:
from IPython.display import Audio
import random

file_id = random.choice([f[:-len(".flac")] for f in flac_files])
directory_1, directory_2, the_file = file_id.split('-')

# Construct the full path
txt_filename = directory_1 + '-' + directory_2
txt_file_path = os.path.join(base_dir, directory_1, directory_2, f"{txt_filename}.trans.txt")
flac_file_path = os.path.join(base_dir, directory_1, directory_2, f"{file_id}.flac")

print("Text Transcription:", read_txt_file(txt_file_path), "\nAudio:")
Audio(filename=flac_file_path)

Text Transcription: {'426-122821-0000': "TEACHING IS REALLY VERY INTERESTING WORK WROTE ANNE TO A QUEEN'S ACADEMY CHUM JANE SAYS SHE THINKS IT IS MONOTONOUS BUT I DON'T FIND IT SO SOMETHING FUNNY IS ALMOST SURE TO HAPPEN EVERY DAY AND THE CHILDREN SAY SUCH AMUSING THINGS", '426-122821-0001': "IT WAS BECAUSE JIMMY CALLED HIM SAINT CLAIR THAT SAINT CLAIR POUNDED HIM ON THE WAY HOME FROM SCHOOL I HEARD OF THE POUNDING BUT NOT OFFICIALLY SO I DON'T THINK I'LL TAKE ANY NOTICE OF IT YESTERDAY I WAS TRYING TO TEACH LOTTIE WRIGHT TO DO ADDITION I SAID", '426-122821-0002': "BECAUSE IT WOULD RAIN THE NEXT DAY IT'S SO HARD NOT TO LAUGH STELLA I HAVE TO SAVE UP ALL MY AMUSEMENT UNTIL I GET HOME AND MARILLA SAYS IT MAKES HER NERVOUS TO HEAR WILD SHRIEKS OF MIRTH PROCEEDING FROM THE EAST GABLE WITHOUT ANY APPARENT CAUSE", '426-122821-0003': 'DID YOU KNOW THAT THOMAS A BECKET WAS CANONIZED AS A SNAKE ROSE BELL SAYS HE WAS ALSO THAT WILLIAM TYNDALE WROTE THE NEW TESTAMENT CLAUDE WHITE SAYS A GLACIER I

In [233]:
def fetch_sound_text_mapping(base_dir):
  for subdir, dirs, files in os.walk(base_dir):
    flac_files = [os.path.join(subdir, f) for f in files if f.endswith(".flac")]
    txt_files = [os.path.join(subdir, f) for f in files if f.endswith(".txt")]

  txt_samples = {}
  for f in txt_files:
    txt_samples.update(read_txt_file(f))

  speech_samples = {}
  for f in flac_files:
    speech_samples.update(read_flac_file(f))

  assert len(txt_samples) == len(speech_samples)

  samples = [(speech_samples[file_id], txt_samples[file_id]) for file_id in speech_samples.keys() if len(speech_samples[file_id]) < audio_maxlen]
  return samples

In [234]:
# Look at some samples

samples = fetch_sound_text_mapping(base_dir)
samples[:5]

[(array([ 3.29589844e-03,  3.32641602e-03,  3.41796875e-03, ...,
         -9.15527344e-05, -6.71386719e-04, -1.22070312e-04]),
  'WHICH APPEARED TO HIM SO EXTRAORDINARY'),
 (array([0.00112915, 0.00186157, 0.00183105, ..., 0.00140381, 0.00164795,
         0.0017395 ]),
  'WE ARE ALL OVERJOYED SAID SHE AT YOUR RETURN BUT YOU ALARM US BY YOUR LAMENTATIONS PRAY TELL US THE CAUSE OF YOUR SORROW ALAS REPLIED THE HUSBAND I HAVE BUT A YEAR TO LIVE'),
 (array([-0.00085449, -0.00100708, -0.0010376 , ...,  0.00241089,
          0.0022583 ,  0.00180054]),
  'BECAUSE HE HAD A GREAT DESERT TO PASS OVER WHERE HE COULD PROCURE NO SORT OF PROVISIONS HE ARRIVED WITHOUT ANY ACCIDENT AT THE END OF HIS JOURNEY AND HAVING DISPATCHED HIS AFFAIRS TOOK HORSE AGAIN IN ORDER TO RETURN HOME'),
 (array([ 0.00341797,  0.003479  ,  0.00360107, ..., -0.00158691,
         -0.00216675, -0.00213623]),
  'BEING A GOOD MOOSULMAUN HE WASHED HIS HANDS FACE AND FEET AND SAID HIS PRAYERS BEFORE HE HAD FINISHED AND WHILE HE WA

In [235]:
print(len(samples))

42


# Pre-process Data

In [195]:
from wav2vec2 import Wav2Vec2Processor
tokenizer = Wav2Vec2Processor(is_tokenizer=True)
processor = Wav2Vec2Processor(is_tokenizer=False)

def preprocess_text(text):
  label = tokenizer(text)
  return tf.constant(label, dtype=tf.int32)

def preprocess_speech(audio):
  audio = tf.constant(audio, dtype=tf.float32)
  return processor(tf.transpose(audio))

In [196]:
def inputs_generator():
  for speech, text in samples:
    yield preprocess_speech(speech), preprocess_text(text)

# Setting up tf.data.Dataset

In [197]:
output_signature = (
    tf.TensorSpec(shape=(None),  dtype=tf.float32),
    tf.TensorSpec(shape=(None), dtype=tf.int32),
)

dataset = tf.data.Dataset.from_generator(inputs_generator, output_signature=output_signature)

In [198]:
buffer_size = len(flac_files)
SEED = 88

dataset = dataset.shuffle(buffer_size, seed=SEED)

In [199]:
dataset = dataset.padded_batch(batch_size, padded_shapes=(audio_maxlen, label_maxlen), padding_values=(0.0, 0))

In [200]:
dataset = dataset.prefetch(tf.data.AUTOTUNE)

In [224]:
num_train_batches = 10
num_val_batches = 4

train_dataset = dataset.take(num_train_batches)
val_dataset = dataset.skip(num_train_batches).take(num_val_batches)

# Model Training

In [225]:
model.compile(optimizer,
              loss=loss_fn)

In [226]:
history = model.fit(train_dataset,
                    validation_data=val_dataset,
                    epochs=10)
history.history

Epoch 1/10


16/16 [==============================] - 36s 2s/step - loss: 442.8341
Epoch 2/10
16/16 [==============================] - 26s 2s/step - loss: 455.0002
Epoch 3/10
16/16 [==============================] - 26s 2s/step - loss: 457.6909
Epoch 4/10
16/16 [==============================] - 29s 2s/step - loss: 454.1705
Epoch 5/10
16/16 [==============================] - 26s 2s/step - loss: 441.1490
Epoch 6/10
16/16 [==============================] - 33s 2s/step - loss: 438.2045
Epoch 7/10
16/16 [==============================] - 26s 2s/step - loss: 416.9953
Epoch 8/10
16/16 [==============================] - 26s 2s/step - loss: 463.8383
Epoch 9/10
16/16 [==============================] - 26s 2s/step - loss: 473.0385
Epoch 10/10
16/16 [==============================] - 30s 2s/step - loss: 462.8322


{'loss': [442.8341369628906,
  455.000244140625,
  457.6908874511719,
  454.1705322265625,
  441.1490478515625,
  438.20452880859375,
  416.9952697753906,
  463.8382568359375,
  473.03851318359375,
  462.8321838378906]}

In [229]:
save_dir = "finetuned-wav2vec2"
model.save(save_dir, include_optimizer=False)

# Evaluation

In [210]:
!pip3 install -q datasets

from datasets import load_metric
metric = load_metric("wer")

In [211]:
@tf.function(jit_compile=True)
def eval_fwd(batch):
  logits = model(batch, training=False)
  return tf.argmax(logits, axis=-1)

In [212]:
from tqdm.auto import tqdm

for speech, labels in tqdm(val_dataset, total=num_val_batches):
    predictions  = eval_fwd(speech)
    predictions = [tokenizer.decode(pred) for pred in predictions.numpy().tolist()]
    references = [tokenizer.decode(label, group_tokens=False) for label in labels.numpy().tolist()]
    metric.add_batch(references=references, predictions=predictions)

  0%|          | 0/4 [00:00<?, ?it/s]

In [213]:
metric.compute()

1.0

# Inference

In [214]:
finetuned_model = tf.keras.models.load_model(save_dir)

In [39]:
!wget https://github.com/vasudevgupta7/gsoc-wav2vec2/raw/main/data/SA2.wav

--2023-12-14 17:17:52--  https://github.com/vasudevgupta7/gsoc-wav2vec2/raw/main/data/SA2.wav
Resolving github.com (github.com)... 140.82.114.4
Connecting to github.com (github.com)|140.82.114.4|:443... connected.
HTTP request sent, awaiting response... 301 Moved Permanently
Location: https://github.com/thevasudevgupta/gsoc-wav2vec2/raw/main/data/SA2.wav [following]
--2023-12-14 17:17:52--  https://github.com/thevasudevgupta/gsoc-wav2vec2/raw/main/data/SA2.wav
Reusing existing connection to github.com:443.
HTTP request sent, awaiting response... 302 Found
Location: https://raw.githubusercontent.com/thevasudevgupta/gsoc-wav2vec2/main/data/SA2.wav [following]
--2023-12-14 17:17:52--  https://raw.githubusercontent.com/thevasudevgupta/gsoc-wav2vec2/main/data/SA2.wav
Resolving raw.githubusercontent.com (raw.githubusercontent.com)... 185.199.111.133, 185.199.109.133, 185.199.108.133, ...
Connecting to raw.githubusercontent.com (raw.githubusercontent.com)|185.199.111.133|:443... connected.
HT

In [215]:
import numpy as np

speech, _ = sf.read("SA2.wav")
speech = np.pad(speech, (0, audio_maxlen - len(speech)))
speech = tf.expand_dims(processor(tf.constant(speech)), 0)

outputs = finetuned_model(speech)
outputs

<tf.Tensor: shape=(1, 768, 32), dtype=float32, numpy=
array([[[ 6.2031255, -2.0537295, -1.6457949, ..., -1.9120979,
         -2.1014853, -2.3190267],
        [ 6.2032385, -2.0539205, -1.6457106, ..., -1.9120598,
         -2.1014917, -2.3188708],
        [ 6.2029977, -2.0538642, -1.6460062, ..., -1.9118725,
         -2.1013935, -2.3189259],
        ...,
        [ 6.2032495, -2.053229 , -1.6449373, ..., -1.911956 ,
         -2.1006892, -2.3193634],
        [ 6.2032833, -2.0532365, -1.6449265, ..., -1.9119612,
         -2.1006956, -2.319378 ],
        [ 6.203337 , -2.0532486, -1.6449959, ..., -1.912048 ,
         -2.1007113, -2.3193965]]], dtype=float32)>

In [216]:
predictions = tf.argmax(outputs, axis=-1)
predictions = [tokenizer.decode(pred) for pred in predictions.numpy().tolist()]
predictions

['']

In [228]:
# Upload a pre-recorded audio file to Colab
from google.colab import files
import numpy as np

uploaded = files.upload()

# Load the uploaded audio file
uploaded_filename = list(uploaded.keys())[0]
speech, _ = sf.read(uploaded_filename)
speech = np.pad(speech, (0, audio_maxlen - len(speech)))
speech = tf.expand_dims(processor(tf.constant(speech)), 0)

outputs = finetuned_model(speech)
predictions = tf.argmax(outputs, axis=-1)
predictions = [tokenizer.decode(pred) for pred in predictions.numpy().tolist()]
predictions

Saving data_SA2.wav to data_SA2 (2).wav


['']